<a href="https://colab.research.google.com/github/stephfz/analisis_codiv_pe/blob/master/TallerPandasWiT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción
¿Qué es Pandas? Es la librería de Python para el Análisis de Datos o Python Data Analysis Library

Para la sesión de hoy usaremos el dataset de [Datos Abiertos de Covid-19](https://www.datosabiertos.gob.pe/group/datos-abiertos-de-covid-19)

# Dataset: Casos Positivos

In [0]:
import pandas as pd
import requests
import io 

url = 'https://raw.githubusercontent.com/stephfz/analisis_codiv_pe/master/datos/positivos_covid.csv'
s = requests.get(url).content

# creamos una variable que se convertira en el contenedor de la data = dataframe
casos_positivos_df = pd.read_csv(io.StringIO(s.decode('latin1')))

casos_positivos_df.head(5)

In [0]:
## Dimension del dataframe
casos_positivos_df.shape

In [0]:
## Revisión rápida de los tipos de cada columna 
## qué tipos de datos tiene pandas? https://pbpython.com/pandas_dtypes.html
casos_positivos_df.dtypes

In [0]:
## Listemos las columnas del dataframe
casos_positivos_df.columns

In [0]:
## Veamos la cantidad de valores nulos por columna del dataframe
columns = casos_positivos_df.columns.tolist()
for column in columns:
  print('Valores nulos para la columna <{0}> : {1}'.format(column, casos_positivos_df[column].isnull().sum()))

In [0]:
casos_positivos_df.DEPARTAMENTO.unique()

In [0]:
## Revisemos cuantos departamento estan contenidos en el dataframe
len(casos_positivos_df.DEPARTAMENTO.unique())

In [0]:
## Revisemos unas cuantos registro del departamento de PUNO
casos_positivos_df[casos_positivos_df.DEPARTAMENTO == 'PUNO'].head(5)

In [0]:
## Veamos los resultados por Sexo y Edad
pd.crosstab(casos_positivos_df.DEPARTAMENTO, casos_positivos_df.SEXO)

In [0]:
# Para poder hacer un analisis sobre las edades es mejor definir rangos de edades
#rango_edades = {"A" : 16, "B": 31, "C": 51, "D": 70}  #, "E" >70 }
rango_edades = {"D" : 70, "C": 51, "B": 31, "A": 16} 
for rango in rango_edades:
  print(rango)

In [0]:
## Agregamos una columna para la definicion de rangos de edad
import numpy as np
casos_positivos_df["RangoEdad"] = np.nan
#casos_positivos_df['RangoEdad'].count()

In [0]:
## Iteramos sobre el dataframe y empezamos a guardar el rango de edad por registro
for rango in rango_edades:
  casos_positivos_df['RangoEdad'] = np.where(casos_positivos_df['EDAD'] < rango_edades[rango], rango, casos_positivos_df['RangoEdad'] )

casos_positivos_df['RangoEdad'].head()

In [0]:
## Verificamos que los cambios hayan sido aplicados
casos_positivos_df.head(20)

In [0]:
## Aplicamos la conversion del ultima tramo de rango de edades
casos_positivos_df['RangoEdad'] = np.where(casos_positivos_df['EDAD']>=70, "E", casos_positivos_df['RangoEdad'] )
casos_positivos_df.head(5)

In [0]:
## Verificamos si hay valores nulos en la edad
casos_positivos_df[casos_positivos_df.RangoEdad == 'nan']

In [0]:
## Hagamos una "tabla cruzada" por Departamento y Rango de Edad
pd.crosstab(casos_positivos_df.DEPARTAMENTO, casos_positivos_df.RangoEdad)

In [0]:
## Veamos la lista de Departamentos en el dataframe
departamentos = casos_positivos_df.DEPARTAMENTO.unique()
for d in departamentos:
  print(d)

In [0]:
## Hagamos una iteracion por departamento y ver la distribucion de infectados
## por RangoEdad y Sexo
departamentos = casos_positivos_df.DEPARTAMENTO.unique()
for d in departamentos:
  df_temp = casos_positivos_df[casos_positivos_df.DEPARTAMENTO == d]
  print("*--------- {} ---------*".format(d))
  print(pd.crosstab(df_temp.SEXO, casos_positivos_df.RangoEdad))
  print("\n")

In [0]:
## Hagamos un intento de visualizacion

import matplotlib.pyplot as plt

cs_femenino = casos_positivos_df[casos_positivos_df.SEXO == 'FEMENINO']
cs_masculino = casos_positivos_df[casos_positivos_df.SEXO == 'MASCULINO']
cs_masculino = cs_masculino.dropna(subset =['EDAD'])
cs_femenino = cs_femenino.dropna(subset =['EDAD'])

fig = plt.figure(figsize=(30,15)) #creamos un canvas o figura de 30x10 pixeles

# queremos ver un plot al costado del otro, para esto pensemos en una grilla (celdas)
plt.subplot2grid((2,3),(0,0))
cs_femenino.RangoEdad.value_counts().plot(kind='bar', alpha=0.5)
plt.title('- Totales RangoEdad - FEMENINO')

plt.subplot2grid((2,3),(0,1))
cs_masculino.RangoEdad.value_counts().plot(kind='bar', alpha=0.5)
plt.title('- Totales RangoEdad - MASCULINO')

plt.subplot2grid((2,3),(1,0))
cs_femenino.RangoEdad.value_counts(normalize = True).plot(kind='bar', alpha=0.5)
plt.title('porcentaje total - FEMENINO')

plt.subplot2grid((2,3),(1,1))
cs_masculino.RangoEdad.value_counts(normalize = True).plot(kind='bar', alpha=0.5)
plt.title('porcentaje total - MASCULINO')

plt.show()

# Dataset: Fallecidos por covid

In [0]:

url = 'https://raw.githubusercontent.com/stephfz/analisis_codiv_pe/master/datos/fallecidos_covid.csv'
s = requests.get(url).content

fallecidos_covid_df = pd.read_csv(io.StringIO(s.decode('latin1')))

fallecidos_covid_df.head(5)

In [0]:
## Dimensiones de Dataset
fallecidos_covid_df.shape

In [0]:
## Valores nulos por columna del dataset
columns = fallecidos_covid_df.columns.tolist()
for column in columns:
  print('Valores nulos para la columna <{0}> : {1}'.format(column, fallecidos_covid_df[column].isnull().sum()))

In [0]:
## Veamos como obtener la cuenta de fallecidos por departamento

#fallecidos_covid_df.groupby('DEPARTAMENTO').count()
fallecidos_covid_df.groupby(['DEPARTAMENTO'])['EDAD_DECLARADA'].count().sort_values(ascending=False)

In [0]:
## Veamos algunos indicadores estadisticos del dataframe
fallecidos_covid_df.describe()

In [0]:
rango_edades = {"D" : 70, "C": 51, "B": 31, "A": 16} 
for rango in rango_edades:
  print(rango)

In [0]:
fallecidos_covid_df.head(5)

In [0]:
## Agregamos una columna para la definicion de rangos de edad
import numpy as np
fallecidos_covid_df["RangoEdad"] = np.nan
#casos_positivos_df['RangoEdad'].count()

In [0]:
## Iteramos sobre el dataframe y empezamos a guardar el rango de edad por registro
for rango in rango_edades:
  fallecidos_covid_df['RangoEdad'] = np.where(fallecidos_covid_df['EDAD_DECLARADA'] < rango_edades[rango], rango, fallecidos_covid_df['RangoEdad'] )


fallecidos_covid_df['RangoEdad'].head()

In [0]:
## Transformamos el ultimo rango de edad en el dataframe
fallecidos_covid_df['RangoEdad'] = np.where(fallecidos_covid_df['EDAD_DECLARADA']>=70, "E", fallecidos_covid_df['RangoEdad'] )
fallecidos_covid_df.head(5)

In [0]:
## Veamos la agrupacion de Fallecidos por Edad en cada Departamento
pd.crosstab(fallecidos_covid_df.DEPARTAMENTO, fallecidos_covid_df.RangoEdad)

In [0]:
## Veamos la cantidad de fallecidos por Rango de Edad
fallecidos_covid_df.groupby(['RangoEdad'])['EDAD_DECLARADA'].count().sort_values(ascending=False)

In [0]:
## Veamos la agrupacion de fallecidos por Sexo
pd.crosstab(fallecidos_covid_df.DEPARTAMENTO, fallecidos_covid_df.SEXO)

In [0]:
## Veamos la agrupacion de fallecidos por Sexo y Rango de Edad
pd.crosstab(fallecidos_covid_df.SEXO, fallecidos_covid_df.RangoEdad)

In [0]:
# #¿Cúantos fallecidxs hay por sexo en cada departamento?
pd.crosstab(fallecidos_covid_df.DEPARTAMENTO, fallecidos_covid_df.SEXO)

In [0]:
fallecidos_covid_df.shape[0] / casos_positivos_df.shape[0] 

# Dataset: Contratación directa

In [0]:
import pandas as pd
import requests
import io 
url = 'https://raw.githubusercontent.com/stephfz/analisis_codiv_pe/master/datos/contrataciones_directas.csv'
s=requests.get(url).content

contratacion_df = pd.read_csv(io.StringIO(s.decode('latin1')))
contratacion_df.tail(3)

In [0]:
## Veamos la dimension del dataset de contraciones
contratacion_df.shape

In [0]:
## Obtengamos la lista de columnas.
contratacion_columns = contratacion_df.columns.tolist()
contratacion_columns

In [0]:
## Veamos con que tipo de datos contamos
contratacion_df.dtypes

In [0]:
## Cuantos RUCPROVEEDOR unicos hay en el dataset 
len(contratacion_df.RUCPROVEEDOR.unique())

In [0]:
## La columna RUCPROVEEDOR es de tipo de dato int64 lo cual no tiene mucho sentido, hagamos una conversion a string
contratacion_df['RUCPROVEEDOR'] = contratacion_df['RUCPROVEEDOR'].astype(str)

In [0]:
## Veamos el ranking de proveedores
contratacion_df['RUCPROVEEDOR'].value_counts(sort=True, ascending=False)

In [0]:
## Veamos los registros con RUCPROVEEDOR con mayor contrataciones
contratacion_df[contratacion_df['RUCPROVEEDOR'] == '20602777279' ]

In [0]:
## Veamos el monto adjudicado en soles por departamento, pero antes..

## veamos con que tipo de datos contamos
contratacion_df.dtypes

In [0]:
contratacion_df['MONTOADJUDICADOSOLES'].head(10)

In [0]:
## Reemplazamos el ',' por '.' para poder parsear los valores como numeros flotantes
contratacion_df['MONTOADJUDICADOSOLES'] = contratacion_df['MONTOADJUDICADOSOLES'].str.replace(',','.')

In [0]:
## Convertimos la cadena a numero float
contratacion_df['MONTOADJUDICADOSOLES'] = contratacion_df['MONTOADJUDICADOSOLES'].astype(float)

In [0]:
## Agrupamos para ver los montos por departamentos
#contratacion_df.groupby(['ENTIDAD_DEPARTAMENTO'])['MONTOADJUDICADOSOLES'].sum()

# funciones anonimas en python: lambda
contratacion_df.groupby(['ENTIDAD_DEPARTAMENTO'])['MONTOADJUDICADOSOLES'].sum().apply(lambda x: "{:,.1f}k".format((x/1000)))

In [0]:
## ¿Cual es el monto total del proveedor con mas adjudicaciones? 
monto = contratacion_df[contratacion_df['RUCPROVEEDOR'] == '20602777279' ]['MONTOADJUDICADOSOLES'].sum()
print ( "{:,.1f}".format((monto)))

# Cruzando datos de diferentes datasets

| DEPARTAMENTO | SEXO | EDAD | INFECTADO O FALLECIDO | RANGOEDAD

In [0]:
## Repasemos los datos que tenemos
casos_positivos_df.head(5)

In [0]:
fallecidos_covid_df.head(5)

In [0]:
## Creamos un subset de los datos de casos positivos
positivos = casos_positivos_df[['DEPARTAMENTO', 'SEXO', 'EDAD', 'RangoEdad']]
## Agrego una columna para el estado de la persona I : Infectado
import numpy as np
positivos["ESTADO"] = "I"
positivos.head()

In [0]:
fallecidos_covid_df.columns

In [0]:
## Creamos un subset de los datos de los fallecidos
fallecidos = fallecidos_covid_df[['DEPARTAMENTO', 'EDAD_DECLARADA', 'SEXO','RangoEdad']]
## Agrego una columna para el estado de la persona F : Fallecido
import numpy as np
fallecidos["ESTADO"] = "F"
fallecidos.head()

In [0]:
## Renombramos las columnas para que los nombres concuerden entre los 2 datasets
fallecidos = fallecidos.rename(columns={"EDAD_DECLARADA": "EDAD"})
fallecidos.head(5)

In [0]:
## Concatenamos los dos dataset para tener un total por departamento y estado (Fallecido, Infectado)
frames = [fallecidos,positivos]
total = pd.concat(frames)
total.tail(10)

In [0]:
## Veamos los datos de Infectados y Fallecidos por Departamento
pd.crosstab(total.DEPARTAMENTO, total.ESTADO).sort_values('F', ascending = False)

In [0]:
total.head(10)

In [0]:
femenino = total[total['SEXO']=='FEMENINO']

#pd.crosstab(femenino.DEPARTAMENTO, femenino.ESTADO).sort_values('F', ascending = False)
pd.crosstab(femenino.ESTADO, femenino.DEPARTAMENTO)

In [0]:
masculino = total[total['SEXO']=='MASCULINO']
#pd.crosstab(masculino.DEPARTAMENTO, masculino.ESTADO).sort_values('F', ascending = False)
pd.crosstab(masculino.ESTADO, masculino.DEPARTAMENTO)

In [0]:
## Analicemos los datos agrupados por sexo y estado por departamento
pd.set_option('display.max_rows', 500)
group = total.groupby(['DEPARTAMENTO','SEXO','ESTADO'])['ESTADO'].count()
print(group)